In [1]:
%load_ext autoreload

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np
from tensorflow import keras
from oodeel.methods import MLS, DKNN, ODIN
from oodeel.methods import DKNN
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from oodeel.eval.metrics import bench_metrics, get_curve
from oodeel.datasets import OODDataset
from oodeel.models.training_funs import train_convnet_classifier

from sklearn.metrics import *

import warnings
warnings.filterwarnings("ignore")

model_path = os.path.expanduser("~/") + ".oodeel/saved_models"
os.makedirs(model_path, exist_ok=True)

/Users/paul/workspace/oodeel/oodeel_dev_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Two datasets experiment

In [2]:
%autoreload 2

oods_in = OODDataset('mnist', split="test")
oods_out = OODDataset('fashion_mnist', split="test")
oods_fit = OODDataset('mnist', split="train")

def preprocess_fn(*inputs):
    x = inputs[0] / 255
    return tuple([x] + list(inputs[1:]))


batch_size = 128
ds_in = oods_in.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn)
ds_out = oods_out.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn)
ds_fit = oods_fit.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn, shuffle=True)

Metal device set to: AMD Radeon Pro 5500M

systemMemory: 64.00 GB
maxCacheSize: 3.99 GB

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
model_path_mnist = os.path.join(model_path, "mnist_model")

try:
    model = tf.keras.models.load_model(model_path_mnist)
    
except OSError:
    train_config = {
        "input_shape": (28, 28, 1),
        "num_classes": 10,
        "batch_size": 128,
        "is_prepared": True,
        "epochs": 5,
        "save_dir": model_path_mnist,
        "validation_data": ds_in#.get_dataset() #ds_in is actually the test set of MNIST
    }

    model = train_convnet_classifier(ds_fit, **train_config) #ds_fit is actually the train set of MNIST

Epoch 1/5
469/469 [==============================] - ETA: 0s - loss: 0.3286 - accuracy: 0.9001

INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/mnist_model/assets


INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/mnist_model/assets


469/469 [==============================] - 18s 29ms/step - loss: 0.3286 - accuracy: 0.9001 - val_loss: 0.0920 - val_accuracy: 0.9722
Epoch 2/5
469/469 [==============================] - ETA: 0s - loss: 0.1048 - accuracy: 0.9687

INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/mnist_model/assets


INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/mnist_model/assets


469/469 [==============================] - 13s 27ms/step - loss: 0.1048 - accuracy: 0.9687 - val_loss: 0.0616 - val_accuracy: 0.9803
Epoch 3/5
468/469 [============================>.] - ETA: 0s - loss: 0.0830 - accuracy: 0.9758

INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/mnist_model/assets


INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/mnist_model/assets


469/469 [==============================] - 12s 25ms/step - loss: 0.0830 - accuracy: 0.9758 - val_loss: 0.0580 - val_accuracy: 0.9822
Epoch 4/5
469/469 [==============================] - 9s 20ms/step - loss: 0.0794 - accuracy: 0.9757 - val_loss: 0.0574 - val_accuracy: 0.9820
Epoch 5/5
469/469 [==============================] - 10s 20ms/step - loss: 0.0791 - accuracy: 0.9758 - val_loss: 0.0572 - val_accuracy: 0.9820


### MLS

In [4]:
oodmodel = MLS()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

metrics = pd.Series(metrics)
print(metrics)


auroc             0.9694
fpr95tpr          0.1093
accuracy_score    0.5000
roc_auc_score     0.9694
dtype: float64


### DKNN

In [5]:
%autoreload 2


oodmodel = DKNN()
oodmodel.fit(model, ds_fit.take(10000))
scores_in = oodmodel.score(ds_in.take(1000))
scores_out = oodmodel.score(ds_out.take(1000))


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

metrics = pd.Series(metrics)
print(metrics)

auroc             0.998736
fpr95tpr          0.005200
accuracy_score    0.500000
roc_auc_score     0.998736
dtype: float64


### ODIN

In [6]:

%autoreload 2
from oodeel.methods import ODIN

oodmodel = ODIN()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

metrics = pd.Series(metrics)
print(metrics)

auroc             0.970709
fpr95tpr          0.107900
accuracy_score    0.500000
roc_auc_score     0.970709
dtype: float64


## Single dataset experiment

(Leave-$k$-classes-out training).
First, we need to define a training function

In [7]:

%autoreload 2

oods_test = OODDataset('mnist', split="test")
oods_train = OODDataset('mnist', split="train")

batch_size = 128
inc_labels = [0, 1, 2, 3, 4]
oods_train, _ = oods_train.assign_ood_labels_by_class(in_labels=inc_labels)
oods_in, oods_out = oods_test.assign_ood_labels_by_class(in_labels=inc_labels)


def preprocess_fn(*inputs):
    x = inputs[0] / 255
    return tuple([x] + list(inputs[1:]))

ds_train = oods_train.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn, shuffle=True)
ds_in = oods_in.prepare(batch_size=batch_size, with_ood_labels=False, preprocess_fn=preprocess_fn)
ds_out = oods_out.prepare(batch_size=batch_size, with_ood_labels=False, preprocess_fn=preprocess_fn)


In [8]:
model_path_mnist_04 = os.path.join(model_path, "mnist_model_0-4/")

try:
    model = tf.keras.models.load_model(model_path_mnist_04)
    
except OSError:
    train_config = {
        "input_shape": (28, 28, 1),
        "num_classes": 10,
        "is_prepared": True,
        "batch_size": 128,
        "epochs": 5,
        "save_dir": model_path_mnist_04,
        "validation_data": ds_in #ds_in is actually the test set of MNIST
    }

    model = train_convnet_classifier(ds_train, **train_config) #ds_fit is actually the train set of MNIST

Epoch 1/5
469/469 [==============================] - ETA: 0s - loss: 0.3411 - accuracy: 0.8960

INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/saved_models/mnist_model_0-4/assets


INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/saved_models/mnist_model_0-4/assets


469/469 [==============================] - 22s 45ms/step - loss: 0.3411 - accuracy: 0.8960 - val_loss: 0.0694 - val_accuracy: 0.9799
Epoch 2/5
469/469 [==============================] - ETA: 0s - loss: 0.1053 - accuracy: 0.9679

INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/saved_models/mnist_model_0-4/assets


INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/saved_models/mnist_model_0-4/assets


469/469 [==============================] - 14s 30ms/step - loss: 0.1053 - accuracy: 0.9679 - val_loss: 0.0550 - val_accuracy: 0.9836
Epoch 3/5
469/469 [==============================] - ETA: 0s - loss: 0.0836 - accuracy: 0.9751

INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/saved_models/mnist_model_0-4/assets


INFO:tensorflow:Assets written to: /Users/paul/.oodeel/saved_models/saved_models/mnist_model_0-4/assets


469/469 [==============================] - 14s 29ms/step - loss: 0.0836 - accuracy: 0.9751 - val_loss: 0.0513 - val_accuracy: 0.9848
Epoch 4/5
469/469 [==============================] - 13s 27ms/step - loss: 0.0814 - accuracy: 0.9750 - val_loss: 0.0513 - val_accuracy: 0.9848
Epoch 5/5
469/469 [==============================] - 11s 24ms/step - loss: 0.0792 - accuracy: 0.9754 - val_loss: 0.0513 - val_accuracy: 0.9848


## MLS

In [9]:
%autoreload 2

oodmodel = MLS()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

metrics = pd.Series(metrics)
print(metrics)

auroc             0.564756
fpr95tpr          0.931893
accuracy_score    0.486100
roc_auc_score     0.564755
dtype: float64


### DKNN

In [10]:
%autoreload 2

oodmodel = DKNN()
oodmodel.fit(model, ds_train.take(10000))
scores_in = oodmodel.score(ds_in.take(1000))
scores_out = oodmodel.score(ds_out.take(1000))


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

metrics = pd.Series(metrics)
print(metrics)

auroc             0.975972
fpr95tpr          0.107803
accuracy_score    0.486100
roc_auc_score     0.975973
dtype: float64


### ODIN

In [11]:
%autoreload 2

#x_test, y_id = data_handler.convert_to_numpy(x_id)

oodmodel = ODIN()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

metrics = pd.Series(metrics)
print(metrics)

auroc             0.558197
fpr95tpr          0.615100
accuracy_score    0.486100
roc_auc_score     0.558197
dtype: float64
